In [1]:
# import packages 
from skimage import io, transform
import numpy as np
import open3d as o3d
import copy
import napari
import time
import os

from skimage import morphology
from skimage.measure import label, regionprops, block_reduce
from scipy import stats, ndimage
import matplotlib.pyplot as plt
import sys

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)


from src.preprocess import segmentation_with_optimized_thresh, image_padding, clean_up_segmented_image
from src.registration import image_to_pcd, pcd_to_image

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
t0 = time.time()

# Read the image, only GFP channel is required:
reference_fly_filename = "/media/ceolin/Data/Lab Gompel/Projects/Fly_Abdomens/data_2/References_and_masks/C1-Reference_Abdomen.tif"
image = io.imread(reference_fly_filename)

# Downscaling:
downscaling =  (2,2,2)
image_downscaled = transform.downscale_local_mean(image, downscaling)[1:-2,1:-2,1:-2]
#image_downscaled = image

# Segment:
Thresholded_Image = segmentation_with_optimized_thresh(image_downscaled, fraction_range = [0.010, 0.015])
#viewer = napari.view_image(thresholded)

# Padding:
Thresholded_Image  = image_padding(Thresholded_Image)

# Clean up the segmentation with morphological transformations:
Thresholded_Image = clean_up_segmented_image(Thresholded_Image)

t1 = time.time()

# print running time:

print("Preprocessing of the image took: ", t1-t0, "seconds")

38
0.014859255378515065
Preprocessing of the image took:  19.20512318611145 seconds


In [3]:
viewer = napari.view_image(Thresholded_Image)

image_padded = image_padding(image_downscaled)

viewer = napari.view_image(Thresholded_Image*image_padded)

In [4]:
# Convert to a pcd to rotate and align the image:

def draw_pcd(pcd):
    pcd_temp = copy.deepcopy(pcd)
    pcd_temp.paint_uniform_color([1, 0.706, 0])
    o3d.visualization.draw_geometries([pcd_temp])

# Create a pcd:
pcd, pcd_original_values  = image_to_pcd(Thresholded_Image*image_padded)

# Translate and rotate the point cloud:
pcd_mean, pcd_cov = pcd.compute_mean_and_covariance()
_ , pcd_eigv = np.linalg.eig(pcd_cov)

# Transform to a base given by the first eigenvector (a,b,c) which should be aligned with the abdomen,
# the vertical versor (0,0,1) and their vector product: (b,-a,0)

b = pcd_eigv[1,0]
c = pcd_eigv[2,0]
b, c = b/(b**2+c**2)**0.5, c/(b**2+c**2)**0.5

temp = np.asarray([[1.0, 0.0, 0.0], [0.0, b, c], [0.0, c, -b]])
transformation_pcd = np.eye(4)
transformation_pcd[:3, :3] = np.linalg.inv(temp)
    
pcd.translate(-pcd_mean)
pcd.transform(transformation_pcd)

array = np.asarray(pcd.points)

translation = [-min(array[:,0])+10,-min(array[:,1])+20, -min(array[:,2])+10]
pcd.translate(translation)
array = np.asarray(pcd.points)

new_reference_colored = pcd_to_image(pcd, pcd_original_values, Thresholded_Image.shape)
new_reference_binary = new_reference_colored > 0

viewer = napari.view_image(new_reference_binary)

In [5]:
# Save the new reference as tiff file
from tifffile import imsave
output_file_name = "/media/ceolin/Data/Lab Gompel/Projects/Fly_Abdomens/data_2/References_and_masks/Reference_abdomen_2_2_2.tif"
imsave(output_file_name, new_reference_colored)

output_file_name = "/media/ceolin/Data/Lab Gompel/Projects/Fly_Abdomens/data_2/References_and_masks/Reference_abdomen_binary_2_2_2.tif"
imsave(output_file_name, new_reference_binary)